configuration of all the variable  cosmosdb and open AI
 The architecture use changefeed and some method to load data 
 
 Loadpdf , loadword for exemple 
 load json too informatio in the db 

 after use readfeed to generate le vector in a vector db 

 when you look for similarity it will give you the source and id of doc in the source , 

 after send the information to llm open ai and build the answer 
 


In [4]:

import config
import json
import os
import sys
import uuid
import datetime
import glob





from openai import AzureOpenAI
from azure.core.exceptions import AzureError
from azure.cosmos import CosmosClient, PartitionKey
from dotenv import dotenv_values
from azure.cosmos import ThroughputProperties
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# specify the name of the .env file name 
env_name = "example.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)
# Azure Cosmos DB connection details
HOST = config['cosmos_host']
MASTER_KEY = config['cosmos_key']

cosmos_connection_string = config['cosmos_string']

container_name = "ChatMessages"


# Azure OpenAI connection details
openai_endpoint = config['openai_endpoint']
openai_key = config['openai_key']
openai_version = config['openai_version']
openai_embeddings_model = config['openai_embeddings_deployment']
openai_chat_model = config['AZURE_OPENAI_CHAT_MODEL']

dbsource = config['cosmosdbsourcedb'] 
colvector= config['cosmosdbsourcecol']
cosmosdbcolcompletion = config['cosmosdbcolcompletion']
container_name = config['cosmosdbcolcompletion']

# Create the OpenAI client
openai_client = AzureOpenAI(
  api_key = openai_key,  
  api_version = openai_version,  
  azure_endpoint =openai_endpoint 
)


This Python function, [`createvectordb`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), is designed to create a container in an Azure Cosmos DB database with specific indexing and vector embedding policies. Here's a step-by-step breakdown of its operations:

1. **Initialize Cosmos Client**: It starts by creating a [`CosmosClient`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fcosmos_client.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A141%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/azure/cosmos/cosmos_client.py") object using a [`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A27%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and a dictionary containing a `masterKey` as [`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"). This client is used to interact with the Azure Cosmos DB service.

2. **Get Database Client**: The function then retrieves a client for the specific database ([`dbsource`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A42%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) using the [`get_database_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fcosmos_client.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A365%2C%22character%22%3A8%7D%5D ".venv/Lib/site-packages/azure/cosmos/cosmos_client.py") method of the [`CosmosClient`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fcosmos_client.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A141%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/azure/cosmos/cosmos_client.py") object.

3. **Print Collection Name**: It prints the name of the collection (container) to be created or ensured exists.

4. **Define Vector Embedding Policy**: A dictionary named [`vector_embedding_policy`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A7%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") is defined, specifying the settings for vector embeddings. This includes the path (`/embedding`), data type (`float32`), distance function (`cosine`), and dimensions (`1536`) of the embeddings.

5. **Define Indexing Policy**: Another dictionary, [`indexing_policy`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A8%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), is defined to specify which paths are included and excluded in the indexing, as well as the vector indexes configuration. It includes all paths (`/*`) except the `_etag` path and specifies a vector index on the `/embedding` path with a type of `quantizedFlat`.

6. **Create or Ensure Container Exists**: The function attempts to create a container (or ensure it exists) in the database client ([`mydbt`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W4sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A3%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) with the specified `id` (collection name), partition key (on the `/id` path), offer throughput settings (with auto-scaling up to 1000 throughput units), and the previously defined indexing and vector embedding policies. If the container is successfully created or confirmed to exist, it prints "container created".

7. **Error Handling**: If an error occurs during the container creation process, the function will raise the caught exception to be handled by the caller.

This function is tailored for scenarios where there's a need to store and efficiently query large sets of vector data in Azure Cosmos DB, leveraging Cosmos DB's capabilities for NoSQL data storage and advanced indexing features like vector indexing for AI and machine learning applications.

In [5]:
def createvectordb(collection):
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(dbsource)
    
    print(collection)
    
    vector_embedding_policy = { "vectorEmbeddings": [ {  "path": "/embedding",  "dataType": "float32",  "distanceFunction": "cosine",  "dimensions": 1536  } ] }
    indexing_policy = { "includedPaths": [ { "path": "/*" } ], "excludedPaths": [  {  "path": "/\"_etag\"/?" }  ], "vectorIndexes": [ {"path": "/embedding", "type": "quantizedFlat"  }] }

    try:
        container = mydbt.create_container_if_not_exists( 
        id= collection, 
        partition_key=PartitionKey(path='/id'), 
        offer_throughput=ThroughputProperties(auto_scale_max_throughput=1000, auto_scale_increment_percent=0),
        indexing_policy=indexing_policy, 
        vector_embedding_policy=vector_embedding_policy) 
        print("container created")
              
    except : 
         raise

This function, [`generate_embeddings`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), takes two parameters: [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A24%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and [`text`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"). It is designed to generate embeddings for the given [`text`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") using the OpenAI API version 1.x. Here's a step-by-step description of what it does:

1. It calls the `embeddings.create` method on the [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A24%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") object. This method is passed two arguments:
   - `input`: The text for which embeddings are to be generated.
   - `model`: A variable [`openai_embeddings_model`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A39%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") that presumably specifies the model to be used for generating embeddings. This variable is not defined within the function, so it must be defined elsewhere in the code or passed into the function's scope in some other way.

2. The response from the `embeddings.create` method is expected to be an object that contains the generated embeddings within its `data` attribute. The code assumes that the response's `data` attribute is a list, and it accesses the first item of this list with `[0]`.

3. From this first item, it accesses the `embedding` attribute to retrieve the actual embeddings.

4. Finally, the function returns the retrieved embeddings.

This function is specifically tailored for use with the OpenAI API and requires an [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A24%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") object that is properly authenticated and configured to interact with the OpenAI API. The function assumes that the API's response is structured in a particular way, which is important to note for maintenance and debugging purposes.

In [6]:
def generate_embeddings(openai_client, text):
    """
    Generates embeddings for a given text using the OpenAI API v1.x
    """
    response = openai_client.embeddings.create(
        input = text,
        model= openai_embeddings_model
    )
    
    embeddings = response.data[0].embedding
    return embeddings



This Python function, [`loaddata`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), is designed to interact with an Azure Cosmos DB instance. It performs several operations as outlined below:

1. **Initialization**: It takes three parameters: [`db`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A13%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (the database name), [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A16%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (the collection or container name within the database), and [`file`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22external%22%3A%22vscode-notebook-cell%3A%2Fc%253A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%23W6sZmlsZQ%253D%253D%22%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (the path to a file containing data to be loaded into the database).

2. **Client Setup**: It initializes a CosmosClient object using a [`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A27%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and a [`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), which are presumably defined elsewhere in the code. This client is used to interact with the Cosmos DB service.

3. **Database and Container Access**: It obtains a reference to the specified database ([`db`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A13%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) and attempts to create a new container within this database if one does not already exist with the specified [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A16%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") name. The container is configured with a partition key on the path `/id` and auto-scalable throughput settings (starting with a maximum throughput of 1000 RU/s and an unspecified increment percentage).

4. **Data Loading**:
   - Opens the specified [`file`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22external%22%3A%22vscode-notebook-cell%3A%2Fc%253A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%23W6sZmlsZQ%253D%253D%22%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (assumed to be in JSON format) and reads its contents.
   - Iterates over the documents (assumed to be a list of JSON objects) in the file.
   - For each document, it upserts (updates or inserts) the document into the container. This means if the document does not exist, it is inserted, and if it does exist (based on some unique identifier, presumably the `/id`), it is updated.

5. **Counting Documents**:
   - Executes a query against the container to count the total number of documents.
   - Iterates over the query results (which should return a single count value) and sums up the counts to get [`total_count`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W6sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A24%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
   - Prints the total count of documents in the container.

6. **Error Handling**: The function uses a broad `try-except` block to catch any exceptions that occur during its execution. If an exception is caught, it simply re-raises the exception without handling it.

This function demonstrates how to interact with Azure Cosmos DB using the CosmosClient from the Azure SDK for Python, including creating containers, upserting documents, and querying data within a container.

In [7]:
def loaddata(db,collection,file) :
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(db)   
  
    print("Creating container..."   )
    


    try:
        container = mydbt.create_container_if_not_exists( 
        id= collection, 
        partition_key=PartitionKey(path='/id'), 
         offer_throughput=ThroughputProperties(auto_scale_max_throughput=1000, auto_scale_increment_percent=0))
        
        filename = file
        with open(filename,encoding="utf8") as file:
            docu = json.load(file)
            for d in docu:
                container.upsert_item(d)
        
# count products
        query = "SELECT VALUE COUNT(1) FROM c"

        total_count = 0

        result = container.query_items(
            query=query,
            enable_cross_partition_query=True)

        for item in result:
            total_count += item

        print("Total count:", total_count)
        
       
    
    except : 
     raise  
       

  

This Python function, [`loadpdffile`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X10sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), performs several operations to load text data from a PDF file into a Cosmos DB collection. Here's a step-by-step breakdown of its functionality:

1. **Initialize Cosmos Client**: It starts by creating a [`CosmosClient`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fcosmos_client.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A141%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/azure/cosmos/cosmos_client.py") instance using a [`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A27%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and a dictionary containing a `masterKey` (presumably for authentication). This client is used to interact with the Cosmos DB.

2. **Get Database Client**: The function then retrieves a client for the specified database ([`db`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X10sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A16%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) using the [`CosmosClient`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fcosmos_client.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A141%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/azure/cosmos/cosmos_client.py") instance.

3. **Load PDF Data**: It uses a [`PyPDFLoader`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2Fdocument_loaders%2Fpdf.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A162%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/langchain_community/document_loaders/pdf.py") (a custom or third-party class not defined in the snippet) to load data from the specified [`file`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X10sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A35%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"). The loaded data is assumed to be text extracted from the PDF.

4. **Split Text Data**: The text data is then split into smaller documents using a [`RecursiveCharacterTextSplitter`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_text_splitters%2Fcharacter.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A57%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/langchain_text_splitters/character.py"), which likely breaks the text into chunks of 1000 characters with an overlap of 150 characters between chunks. This is useful for processing large texts or for fitting into database constraints.

5. **Print First Document**: It prints the first document chunk to the console, likely for debugging or verification purposes.

6. **Create or Get Container**: The function attempts to create a new container in the database if it doesn't exist. The container is identified by [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X10sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A19%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), uses `/id` as the partition key, and is configured with auto-scalable throughput properties.

7. **Insert Documents into Container**: For each document chunk created earlier, it generates a new document with a unique `id` (using [`uuid4`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FLib%2Fuuid.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A722%2C%22character%22%3A4%7D%5D "../../Lib/uuid.py")), the original [`file`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X10sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A35%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") name, and the chunked `text`. Each document is then upserted into the container. Upserting means that the document will be inserted if it doesn't exist or updated if it does.

8. **Error Handling**: The insertion process is wrapped in a try-except block to handle potential errors gracefully. If an error occurs, it raises an exception.

9. **Count Documents**: After inserting the documents, it attempts to count the total number of documents in the container using a SQL query. The total count is printed to the console.

10. **Error Handling for Counting**: The counting process is also wrapped in a try-except block to handle potential errors.

This function is designed to process and store large text data from PDF files into Cosmos DB, making the data queryable and manageable. It includes basic error handling and prints out useful information for debugging.

In [8]:
def loadpdffile(db,collection,name,file) :
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(db)   
    
    loader = PyPDFLoader(file)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(data)
   
    print(docs[0]) 
    
    try:
        container = mydbt.create_container_if_not_exists( 
        id= collection, 
        partition_key=PartitionKey(path='/id'), 
        offer_throughput=ThroughputProperties(auto_scale_max_throughput=1000, auto_scale_increment_percent=0))
        
       
        for d in docs : 
            docu= {} 
            docu["id"] = str(uuid.uuid4())
            docu["file"] = name
            docu["text"] = str(d)
            print(docu)
            container.upsert_item(docu)
    except : 
     raise     
# count 
    try : 
     query = "SELECT VALUE COUNT(1) FROM c"

     total_count = 0

     result = container.query_items(
     query=query,
     enable_cross_partition_query=True)

     for item in result:
        total_count += item

        print("Total count:", total_count)
        
       
    
    except : 
     raise  
       

This Python function, [`loadwordfile`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), performs several operations to load a Word document into a Cosmos DB NoSQL database. Here's a step-by-step breakdown of its functionality:

1. **Initialization**: The function takes four parameters: [`db`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A17%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (database name), [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A20%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (collection name within the database), [`name`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A31%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (a name identifier for the file), and [`file`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A36%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (the path to the Word document file).

2. **CosmosClient Setup**: It initializes a CosmosClient using a predefined [`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A27%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and [`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to connect to the Azure Cosmos DB instance.

3. **Database Client**: Retrieves a client for the specified database ([`db`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A17%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) using the CosmosClient instance.

4. **Document Loader**: Utilizes the [`Docx2txtLoader`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2Fdocument_loaders%2Fword_document.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A16%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/langchain_community/document_loaders/word_document.py") class from the [`langchain_community.document_loaders`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2F__init__.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D ".venv/Lib/site-packages/langchain_community/__init__.py") module to load the content of the Word document specified by the [`file`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A36%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") parameter.

5. **Text Splitting**: Employs a [`RecursiveCharacterTextSplitter`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_text_splitters%2Fcharacter.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A57%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/langchain_text_splitters/character.py") (not imported in the provided code, so assumed to be defined elsewhere or imported implicitly) to split the loaded document text into smaller chunks. This is likely for easier storage and management in the database. The splitter uses a `chunk_size` of 1000 characters and an `overlap` of 150 characters.

6. **Container Creation**: Attempts to create a new container in the database if it does not already exist, using the specified [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A20%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") name. It sets a partition key on the `/id` path and configures auto-scaling throughput properties.

7. **Document Insertion**: Iterates over the chunks of text produced by the text splitter. For each chunk, it creates a new document with a unique `id` (generated using [`uuid.uuid4()`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FLib%2Fuuid.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D "../../Lib/uuid.py")), the [`name`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X11sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A31%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") of the file, and the chunk of text itself. Each document is then upserted into the Cosmos DB container, meaning it will be inserted if it doesn't exist or updated if it does.

8. **Count Query**: After inserting the documents, it executes a query to count the total number of documents in the


In [9]:
def loadwordfile(db,collection,name,file) :
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(db)   
    
    from langchain_community.document_loaders import Docx2txtLoader

    loader = Docx2txtLoader(file)

    data = loader.load()    
    
    
    
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(data)
   
    print(docs[0]) 
    
    try:
        container = mydbt.create_container_if_not_exists( 
        id= collection, 
        partition_key=PartitionKey(path='/id'), 
        offer_throughput=ThroughputProperties(auto_scale_max_throughput=1000, auto_scale_increment_percent=0))
        
       
        for d in docs : 
            docu= {} 
            docu["id"] = str(uuid.uuid4())
            docu["file"] = name
            docu["text"] = str(d)
            print(docu)
            container.upsert_item(docu)
    except : 
     raise     
# count 
    try : 
     query = "SELECT VALUE COUNT(1) FROM c"

     total_count = 0

     result = container.query_items(
     query=query,
     enable_cross_partition_query=True)

     for item in result:
        total_count += item

        print("Total count:", total_count)
        
       
    
    except : 
     raise  
       

The function [`add_doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") is designed to add a document to a specified collection in a NoSQL database, presumably in a Cosmos DB context, given its naming convention. It takes four parameters:

1. [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A12%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): This parameter is likely an instance of a client connected to the OpenAI API, used for generating embeddings.
2. [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): This represents the collection within the database where the document will be added or updated.
3. [`doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): A dictionary representing the document to be added. It must at least contain an "id" key.
4. [`name`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A43%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): A string representing the source of the document, which will be added to the document under the "source" key.

The function performs the following steps:

1. Initializes an empty dictionary [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
2. Copies the "id" from the input [`doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
3. Adds a "source" field to [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), with its value set to the [`name`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A43%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") parameter.
4. Calls a function [`generate_embeddings`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") with the [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A12%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and a JSON string representation of [`doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to generate embeddings. The result is added to [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") under the "embedding" key.
5. Prints the "id" of the document to the console.
6. Attempts to add or update the document in the specified [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") by calling `upsert_item` with [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
7. Catches and prints any exceptions that occur during the process.

This function is useful for processing and storing documents with generated embeddings in a NoSQL database, with error handling to catch and report any issues during the operation.

In [10]:
def add_doc(openai_client, collection, doc,name):
   
  
  
    try:
        doc1 = {}
        doc1["id"] = doc["id"]
        doc1["source"]= name
        doc1["embedding"] = generate_embeddings(openai_client, json.dumps(doc))
        
        print(doc["id"])
        
        collection.upsert_item(doc1)
       
    except Exception as e:
        print(str(e))

This Python function, [`add_doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), is designed to add a document to a specified collection in a NoSQL database, such as Azure Cosmos DB. It takes four parameters:

1. [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A12%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): An instance or client object that interacts with the OpenAI API, presumably used here for generating embeddings.
2. [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The target collection within the NoSQL database where the document will be added or updated.
3. [`doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The document to be added or updated in the collection. It is expected to be a dictionary with at least an "id" key.
4. [`name`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A43%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): A string that seems to be used as a source identifier for the document.

The function performs the following steps:

- Initializes an empty dictionary [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
- Copies the "id" from the input [`doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
- Sets the "source" field in [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to the provided [`name`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A43%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
- Calls a function [`generate_embeddings`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") with the [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A12%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and a JSON string representation of [`doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A39%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to generate embeddings. The result is assigned to the "embedding" field in [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
- Prints the "id" of the document to the console.
- Attempts to add or update [`doc1`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A5%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") in the specified [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A27%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") using the `upsert_item` method, which inserts the item if it does not exist or updates it if it does.
- Catches and prints any exceptions that occur during the process.

This function is useful for processing and storing documents with generated embeddings in a NoSQL database, leveraging the capabilities of an AI model (via [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A12%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) for generating those embeddings.

In [11]:

def get_completion(openai_client, model, prompt: str):    
   
   
    response = openai_client.chat.completions.create(
        model = model,
        messages = [{"role": "user", "content": prompt}]
    )   
   
    return response.choices[0].message.content



This Python function, [`get_similar_docs`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), is designed to retrieve documents from a Cosmos DB for NoSQL that are similar to a given query text. Here's a step-by-step breakdown of its operations:

1. **Function Signature**: The function accepts four parameters:
   - [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A21%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): An instance of a client connected to OpenAI, presumably used for generating embeddings.
   - [`db`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A36%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The name of the database within Cosmos DB.
   - [`query_text`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A40%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): The user's query as a string.
   - [`limit`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A52%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"): An optional parameter specifying the maximum number of documents to return, with a default value of 3.

2. **Vectorize the Question**: It uses the [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A21%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to generate embeddings for the [`query_text`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A40%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"). This process converts the query text into a vector representation that can be used for similarity comparisons.

3. **Query Cosmos DB**:
   - It constructs a SQL-like query to select the top [`limit`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A52%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") documents from a container within the database. These documents are selected based on their similarity to the query vector, calculated using a `VectorDistance` function.
   - The query is executed against the Cosmos DB using the [`query_items`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fazure%2Fcosmos%2Fcontainer.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A365%2C%22character%22%3A8%7D%5D ".venv/Lib/site-packages/azure/cosmos/container.py") method of a container client. This method is configured to allow cross-partition queries.

4. **Filter Results Based on Similarity**: The function iterates over the query results and filters out documents based on a predefined similarity threshold ([`sim = 0.78`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A16%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")). Only documents with a similarity score higher than this threshold are considered.

5. **Retrieve Full Documents**: For each document that passes the similarity threshold, the function constructs a new query to retrieve the full document from the database based on its ID.

6. **Return Results**: The function returns a list of documents that match the query criteria and exceed the similarity threshold. If no documents meet these criteria, an empty list is returned.

**Key Operations**:
- Vectorization of query text for similarity comparison.
- Execution of a Cosmos DB query to find similar documents based on vector similarity.
- Filtering of results based on a similarity threshold.
- Retrieval of full documents from the database.

**Potential Improvements**:
- The function assumes the existence of certain global variables ([`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A27%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`colvector`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A43%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) and functions ([`generate_embeddings`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W5sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) without them being passed as parameters or defined within the function, which could limit reusability and clarity.
- Error handling is not explicitly addressed, which could lead to issues if the database queries fail or if no documents meet the similarity threshold.
- The function's efficiency could potentially be improved by optimizing the database queries or the process of filtering and retrieving documents.

In [16]:
def get_similar_docs(openai_client, db, query_text, limit=3):
    """ 
        Get similar documents from Cosmos DB for NoSQL 

        input: 
            container: name of the container
            query_text: user question
            limit: max number of documents to return
        output:
            documents: json documents similar to the user question
            elapsed_time
    """
    # vectorize the question
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(db)   
    cvector = mydbt.get_container_client(colvector)
    sim = 0.78
   
    query_vector = generate_embeddings(openai_client, query_text)
  
    # find product keys of products that match the question
    query = f"""
        SELECT TOP {limit} 
             c.id , c.source ,VectorDistance(c.embedding, {query_vector}) as similarity
        FROM c
        ORDER BY VectorDistance(c.embedding, {query_vector})
    """
     
    
    results = cvector.query_items(
        query=query,
        parameters=None,
        enable_cross_partition_query=True
    )   
 
           
    listid = []
    source = ""
    # get products from list of id
    id_list = [id for id in results]

    for i in id_list:
        if i['similarity'] > sim:
            listid.append(i['id'])
            source = (i['source'])
            
        
        
 
    
    if listid == []:
        products = []
    else : 
        id_list_str = ', '.join([f"'{id}'" for id in listid]) 
        
      
        mycolt = mydbt.get_container_client(source)
            
        query = f"""
                    SELECT * FROM c 
                    WHERE  c.id IN ({id_list_str})
                """
                
            
        results = mycolt.query_items(
                    query=query,
                    enable_cross_partition_query=True
        )

        products = []
        for product in results:
            products.append(product)    

        
            


    return products


The [`ReadFeed`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X15sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function performs the following operations:

1. **Initialize Cosmos Client**: It creates a CosmosClient instance using a host URL ([`HOST`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A27%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) and a master key ([`MASTER_KEY`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")). This client is used to interact with the Azure Cosmos DB service.

2. **Get Database and Container Clients**: It retrieves clients for a specific database ([`dbsource`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A42%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) and two containers within that database. The first container client is for the container specified by the [`collection`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X15sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A13%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") parameter passed to the function. The second container client is for a container named "vector".

3. **Define Start Time for Reading the Feed**: It calculates a start time for reading the change feed by subtracting one day from the current datetime. This means it will read changes that occurred from one day ago up to the present.

4. **Print Start Times**: It prints the current time and the calculated start time (one day ago) to the console.

5. **Query the Change Feed**: It queries the change feed of the specified collection ([`mycolt`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X15sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A7%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")) starting from the calculated start time. The change feed includes changes to the items in the container, allowing applications to process or react to the data changes.

6. **Process Each Document in the Change Feed**: For each document in the change feed, it calls the [`add_doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function, passing in an [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A48%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), the container client for the "vector" container ([`mycoltembed`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X15sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A8%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")), the document itself, and the name of the collection. This suggests that [`add_doc`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X12sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") is likely responsible for processing or storing each document, possibly involving some interaction with an OpenAI client.

In summary, the [`ReadFeed`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X15sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function is designed to connect to an Azure Cosmos DB instance, read the change feed for a specified container starting from one day ago, and process each document in the feed, potentially involving some operation with OpenAI.

In [13]:
def ReadFeed(collection):
        
       
        
        
        client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
        mydbt = client.get_database_client(dbsource)   
        mycolt = mydbt.get_container_client(collection)
        mycoltembed = mydbt.get_container_client("vector") 
        name = collection
        
     
        # Define a point in time to start reading the feed from
        time = datetime.datetime.now()
        
        print (time)
        time = time - datetime.timedelta(days=1)
        print (time)
        
           
        
        response = mycolt.query_items_change_feed(start_time=time)
        #response = mycolt.query_items_change_feed( )
        
        for doc in response:
            add_doc(openai_client, mycoltembed, doc,name)
           
            
            
        
 

 

Load some data like customer , or insurance sample 

In [ ]:

createvectordb(colvector)
#loaddata(dbsource,'customer',"customer.json")
#ReadFeed('customer')
loaddata(dbsource,'insurance',"insurance.json")
ReadFeed('insurance')







make a test to check 

In [ ]:
get_similar_docs(openai_client,dbsource ,"'customer_segment': 'VIP', 'marital_status", 5)

load some pdf from dataset sample

In [ ]:

# Query 2
# Process a PDF and create embeddings
file_path = "dataset\AXA_Conditions_Generales_Assurance_Habitation.pdf"  # Replace with your actual file path
loadpdffile(dbsource,'pdf','axa',file_path)


file_path = "dataset\Groupama-Resultats-offre-de-rachat-CP.pdf"  # Replace with your actual file path
loadpdffile(dbsource,'pdf','groupama',file_path)
ReadFeed('pdf')


load some word sample

In [ ]:
file_path = "dataset\sample.docx"  # Replace with your actual file path
loadwordfile(dbsource,'word','sample',file_path)
ReadFeed('word')


can make test 

In [ ]:

get_similar_docs(openai_client,dbsource ,"groupama", 5)

get_similar_docs(openai_client,dbsource ,"Assurance habitations", 5)

get_similar_docs(openai_client,dbsource ,"proposal for services", 5)

The provided Python code snippet defines a function named [`slow_echo`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X25sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") and sets up a Gradio chat interface that uses this function. Here's a step-by-step description of the [`slow_echo`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X25sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function and the surrounding context:

1. **Imports and Setup**: The code imports necessary libraries and modules, including [`time`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.7.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Ftime.pyi%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D "../../.vscode/extensions/ms-python.vscode-pylance-2024.7.1/dist/typeshed-fallback/stdlib/time.pyi"), [`gradio`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fgradio%2F__init__.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D ".venv/Lib/site-packages/gradio/__init__.py") (as [`gr`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Fgradio%2F__init__.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D ".venv/Lib/site-packages/gradio/__init__.py")), and [`datetime`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FLib%2Fdatetime.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D "../../Lib/datetime.py"). It also imports [`CosmosDBChatMessageHistory`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2Fchat_message_histories%2Fcosmos_db.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A21%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/langchain_community/chat_message_histories/cosmos_db.py") from [`langchain_community.chat_message_histories.cosmos_db`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2F__init__.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D ".venv/Lib/site-packages/langchain_community/__init__.py").

2. **Cosmos DB Setup**: It initializes a [`CosmosDBChatMessageHistory`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2Fchat_message_histories%2Fcosmos_db.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A21%2C%22character%22%3A6%7D%5D ".venv/Lib/site-packages/langchain_community/chat_message_histories/cosmos_db.py") object with various parameters like [`cosmos_endpoint`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2Fchat_message_histories%2Fcosmos_db.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A26%2C%22character%22%3A8%7D%5D ".venv/Lib/site-packages/langchain_community/chat_message_histories/cosmos_db.py"), [`cosmos_database`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2Fchat_message_histories%2Fcosmos_db.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A27%2C%22character%22%3A8%7D%5D ".venv/Lib/site-packages/langchain_community/chat_message_histories/cosmos_db.py"), [`cosmos_container`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2Fchat_message_histories%2Fcosmos_db.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A8%7D%5D ".venv/Lib/site-packages/langchain_community/chat_message_histories/cosmos_db.py"), [`connection_string`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2Fchat_message_histories%2Fcosmos_db.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A32%2C%22character%22%3A8%7D%5D ".venv/Lib/site-packages/langchain_community/chat_message_histories/cosmos_db.py"), [`session_id`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2Fchat_message_histories%2Fcosmos_db.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A29%2C%22character%22%3A8%7D%5D ".venv/Lib/site-packages/langchain_community/chat_message_histories/cosmos_db.py"), and [`user_id`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2F.venv%2FLib%2Fsite-packages%2Flangchain_community%2Fchat_message_histories%2Fcosmos_db.py%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A30%2C%22character%22%3A8%7D%5D ".venv/Lib/site-packages/langchain_community/chat_message_histories/cosmos_db.py"). These parameters are essential for connecting to and interacting with a Cosmos DB instance.

3. **Function Definition - [`slow_echo`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X25sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb")**:
   - **Parameters**: The function takes two parameters: [`message`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X25sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A14%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (the user's message) and [`history`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X25sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A23%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") (not used within the function).
   - **Processing**: It iterates over each character in the [`message`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X25sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A14%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), simulating a delay ([`time.sleep(0.05)`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2F.vscode%2Fextensions%2Fms-python.vscode-pylance-2024.7.1%2Fdist%2Ftypeshed-fallback%2Fstdlib%2Ftime.pyi%22%2C%22scheme%22%3A%22file%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A0%7D%5D "../../.vscode/extensions/ms-python.vscode-pylance-2024.7.1/dist/typeshed-fallback/stdlib/time.pyi")) to mimic a slow typing effect.
   - **Document Retrieval**: Calls [`get_similar_docs`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X14sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A0%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function with parameters [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A48%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`dbsource`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A42%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`message`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X25sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A14%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), and [`top_k`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X25sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A21%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to retrieve documents similar to the user's message from a database.
   - **User Prompt Creation**: Constructs a user prompt string that includes the context (retrieved documents) and the user's question. This prompt is formatted in markdown and is intended for use with an AI model to generate a response.
   - **Message Logging**: Adds the user's message to the Cosmos DB history using [`cosmos_db_history.add_user_message`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X25sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A12%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
   - **AI Response Generation**: Calls [`get_completion`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X13sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A1%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") with parameters [`openai_client`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A48%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), [`openai_chat_model`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22W2sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A40%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb"), and the constructed [`user_prompt`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X25sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A38%2C%22character%22%3A8%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") to generate an AI response.
   - **AI Message Logging**: Logs the AI's response to the Cosmos DB history using [`cosmos_db_history.add_ai_message`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X25sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A12%2C%22character%22%3A0%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb").
   - **Yielding Response**: Yields a string that prefixes the AI's response with "response: ". This is done for each character in the user's message, due to the loop, but the description suggests this might not align with the intended behavior (likely a mistake in the description or a misunderstanding of the loop's purpose).

4. **Gradio Chat Interface**: Initializes a Gradio chat interface with the [`slow_echo`](command:_github.copilot.openSymbolFromReferences?%5B%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22c%3A%5C%5CUsers%5C%5Cedeletang%5C%5CDocuments%5C%5Cdemonosqlcomsosdb%5C%5Ccosmosddbnosql.ipynb%22%2C%22_sep%22%3A1%2C%22path%22%3A%22%2Fc%3A%2FUsers%2Fedeletang%2FDocuments%2Fdemonosqlcomsosdb%2Fcosmosddbnosql.ipynb%22%2C%22scheme%22%3A%22vscode-notebook-cell%22%2C%22fragment%22%3A%22X25sZmlsZQ%3D%3D%22%7D%2C%7B%22line%22%3A28%2C%22character%22%3A4%7D%5D "c:\Users\edeletang\Documents\demonosqlcomsosdb\cosmosddbnosql.ipynb") function as its backend. This interface allows users to interact with the function through a web-based chat UI.

5. **Main Block**: If the script is run directly (not imported), it launches the Gradio interface, making it accessible for user interaction.

The primary purpose of this code is to simulate a chat interface where users can input messages, and the system, using AI and database queries, generates and displays responses slowly, mimicking a typing effect. The responses are based on the context retrieved from a Cosmos DB instance and processed by an AI model.

In [17]:
import time
import gradio as gr
from datetime import datetime

from langchain_community.chat_message_histories.cosmos_db import CosmosDBChatMessageHistory

# Replace the placeholders with your actual Cosmos DB account details


user = "test"
current_dt = str(datetime.now().strftime("%Y%m%d_%H%M%S"))

cosmos_db_history = CosmosDBChatMessageHistory(
        cosmos_endpoint=HOST,
        cosmos_database=dbsource,
        cosmos_container=container_name,
        connection_string=cosmos_connection_string,
        session_id=current_dt,
        user_id=user
)

top_k = 10

cosmos_db_history.prepare_cosmos()




def slow_echo(message, history):
    for i in range(len(message)):
        time.sleep(0.05)
        
       
        
       
        docs = get_similar_docs(openai_client,dbsource ,message, top_k)
        
        
        user_prompt = f"""
        Using the following CONTEXT, answer the user's question as best as possible. 
            
            - Answer in markdown format

            CONTEXT:
            {docs}

            USER QUESTION:
            {message}
        
        ANSWER:
        """   
        
        cosmos_db_history.add_user_message(message)
        
        response = get_completion(openai_client, openai_chat_model, user_prompt)
        cosmos_db_history.add_ai_message(response)
      
        yield "response: " + response 
      
       
        
        
demo = gr.ChatInterface(slow_echo).queue()

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
